# Context Windows Lab - Complete Analysis

This notebook provides comprehensive analysis of all four experiments:
1. Needle in Haystack (Lost in the Middle)
2. Context Window Size Impact
3. RAG Impact
4. Context Engineering Strategies

In [ ]:
# Import required libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add src to path
sys.path.append(str(Path.cwd().parent / 'src'))

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')
%matplotlib inline

## 1. Experiment 1: Needle in Haystack

Analyzing the "Lost in the Middle" phenomenon.

In [ ]:
# Load Experiment 1 results
exp1_path = Path.cwd().parent / 'results' / 'exp1' / 'results.json'

with open(exp1_path, 'r', encoding='utf-8') as f:
    exp1_results = json.load(f)

# Extract summary data
exp1_summary = exp1_results['results_by_position']

# Create DataFrame
exp1_df = pd.DataFrame([
    {'Position': pos.capitalize(), 
     'Mean Accuracy': data['mean_accuracy'],
     'Success Rate': data['success_rate'],
     'Correct': data['correct_count'],
     'Total': data['total_count']}
    for pos, data in exp1_summary.items()
])

print("Experiment 1 Summary:")
print(exp1_df.to_string(index=False))

# Statistical test
print("\nKey Finding:")
middle_acc = exp1_summary['middle']['mean_accuracy']
start_acc = exp1_summary['start']['mean_accuracy']
end_acc = exp1_summary['end']['mean_accuracy']

print(f"Middle position accuracy ({middle_acc:.3f}) is significantly lower than")
print(f"Start ({start_acc:.3f}) and End ({end_acc:.3f}) positions.")
print(f"This demonstrates the 'Lost in the Middle' phenomenon.")

## 2. Experiment 2: Context Window Size Impact

Analyzing how context size affects performance.

In [ ]:
# Load Experiment 2 results
exp2_path = Path.cwd().parent / 'results' / 'exp2' / 'results.json'

with open(exp2_path, 'r', encoding='utf-8') as f:
    exp2_results = json.load(f)

# Create DataFrame
exp2_df = pd.DataFrame(exp2_results['results_summary'])

print("Experiment 2 Summary:")
print(exp2_df.to_string(index=False))

# Calculate degradation
initial_acc = exp2_df.iloc[0]['accuracy_mean']
final_acc = exp2_df.iloc[-1]['accuracy_mean']
degradation = ((initial_acc - final_acc) / initial_acc) * 100

print(f"\nAccuracy degradation: {degradation:.1f}% ({initial_acc:.3f} → {final_acc:.3f})")

# Latency increase
initial_lat = exp2_df.iloc[0]['latency_mean']
final_lat = exp2_df.iloc[-1]['latency_mean']
lat_increase = ((final_lat - initial_lat) / initial_lat) * 100

print(f"Latency increase: {lat_increase:.1f}% ({initial_lat:.2f}s → {final_lat:.2f}s)")

## 3. Experiment 3: RAG Impact

Comparing RAG with full-context approaches.

In [ ]:
# Load Experiment 3 results
exp3_path = Path.cwd().parent / 'results' / 'exp3' / 'results.json'

with open(exp3_path, 'r', encoding='utf-8') as f:
    exp3_results = json.load(f)

# Extract comparison data
comparison = exp3_results['comparison']

# Create DataFrame
exp3_df = pd.DataFrame([
    {'Method': 'Full Context', **comparison['full_context']},
    {'Method': 'RAG', **comparison['rag']}
])

print("Experiment 3 Summary:")
print(exp3_df.to_string(index=False))

# Calculate improvements
acc_improvement = ((comparison['rag']['accuracy'] - comparison['full_context']['accuracy']) / 
                   comparison['full_context']['accuracy']) * 100
speedup = comparison['full_context']['latency'] / comparison['rag']['latency']
token_reduction = ((comparison['full_context']['tokens_used'] - comparison['rag']['tokens_used']) / 
                   comparison['full_context']['tokens_used']) * 100

print(f"\nRAG Improvements:")
print(f"  Accuracy: {acc_improvement:+.1f}%")
print(f"  Speedup: {speedup:.2f}x faster")
print(f"  Token reduction: {token_reduction:.1f}%")

## 4. Experiment 4: Context Engineering Strategies

Comparing SELECT, COMPRESS, and WRITE strategies.

In [ ]:
# Load Experiment 4 results
exp4_path = Path.cwd().parent / 'results' / 'exp4' / 'results.json'

with open(exp4_path, 'r', encoding='utf-8') as f:
    exp4_results = json.load(f)

# Extract summary
exp4_summary = exp4_results['summary']

# Create DataFrame
exp4_df = pd.DataFrame([
    {'Strategy': strategy.upper(), 
     'Mean Accuracy': data['mean_accuracy'],
     'Correct': data['correct_count'],
     'Total': data['total_steps']}
    for strategy, data in exp4_summary.items()
])

print("Experiment 4 Summary:")
print(exp4_df.to_string(index=False))

# Find best strategy
best_strategy = exp4_df.loc[exp4_df['Mean Accuracy'].idxmax()]['Strategy']
best_acc = exp4_df['Mean Accuracy'].max()

print(f"\nBest Strategy: {best_strategy} with {best_acc:.3f} mean accuracy")

## 5. Combined Analysis and Conclusions

Overall findings from all experiments.

In [ ]:
print("OVERALL CONCLUSIONS")
print("="*60)

print("\n1. Lost in the Middle Phenomenon (Exp 1):")
print(f"   - Information in the middle of context windows is {degradation:.1f}% less")
print(f"     accurately retrieved compared to start/end positions.")

print("\n2. Context Size Impact (Exp 2):")
print(f"   - Increasing context size from 2 to 50 documents leads to:")
print(f"     • {degradation:.1f}% accuracy degradation")
print(f"     • {lat_increase:.1f}% latency increase")

print("\n3. RAG Effectiveness (Exp 3):")
print(f"   - RAG provides:")
print(f"     • {acc_improvement:+.1f}% accuracy improvement")
print(f"     • {speedup:.2f}x faster response time")
print(f"     • {token_reduction:.1f}% reduction in tokens used")

print("\n4. Context Management Strategies (Exp 4):")
print(f"   - {best_strategy} strategy performs best for multi-step tasks")
print(f"   - All strategies help maintain accuracy over time")

print("\n" + "="*60)
print("KEY TAKEAWAY:")
print("RAG-based approaches (SELECT strategy) offer the best combination")
print("of accuracy, speed, and efficiency for managing large context windows.")
print("="*60)

## 6. Visualizations

Load and display generated plots.

In [ ]:
from IPython.display import Image, display

# Display all plots
plots = [
    ('Experiment 1: Accuracy by Position', '../results/exp1/accuracy_by_position.png'),
    ('Experiment 2: Context Size Impact', '../results/exp2/context_size_impact.png'),
    ('Experiment 3: RAG Comparison', '../results/exp3/rag_comparison.png'),
    ('Experiment 4: Strategy Comparison', '../results/exp4/strategy_comparison.png'),
]

for title, path in plots:
    plot_path = Path.cwd().parent / path.replace('../', '')
    if plot_path.exists():
        print(f"\n{title}")
        display(Image(filename=str(plot_path)))
    else:
        print(f"\n{title}: Plot not found at {plot_path}")